## Ball Picking Challenge

### Part 1: Navigation

The aim is to program the robot for the first part of the challenge: follow the line from the beginning to the destination point. To do so, you need to reuse the abilities learnt in weeks 1-3; please feel free to reuse the code of those notebooks and exercises.

In [364]:
# and here starts your job, good luck!

In [365]:
import packages.initialization
import pioneer3dx as p3dx
p3dx.init()
import cv2
import numpy

Found ROS controller /pioneer3dx_28647_ip_172_31_10_144


In [366]:
#%matplotlib inline
#import matplotlib.pyplot as plt

In [367]:
"""
p3dx.tilt(-0.47)
move(0.1,0)
p3dx.sleep(2)
move(0,0)
plt.imshow(p3dx.image);
"""

'\np3dx.tilt(-0.47)\nmove(0.1,0)\np3dx.sleep(2)\nmove(0,0)\nplt.imshow(p3dx.image);\n'

In [368]:
DEF_X_SPEED = 0.22
DEF_YAW_SPEED = 0.34

MIN_WALL_THRESHOLD = 0.15
MAX_WALL_THRESHOLD = 0.2

In [369]:
def is_obstacle_detected():
    Threshold = 0.41
    if min(p3dx.distance[3:5]) < Threshold:
        return True
    else:
        return False

In [370]:
def follow_line():
    print('Following the line')
    width = 150
    Kp = 0.05
    while not is_obstacle_detected():  
        cx, cy, line_end = line_centroid(p3dx.image)
        if line_end == False:
            if cx != None:
                err = cx - width/2
                linear = 0.4
                angular = -Kp*err
                move(linear, angular)
            else:
                move(linear, 0)
        else:
            move(0,0)
    print('Obstacle detected')

In [371]:
def getWall():
    leftSide, frontSide = getSonars()
    # move forward until an obstacle is detected at the front
    while frontSide > MAX_WALL_THRESHOLD:       
        move(DEF_X_SPEED,0)
        leftSide, frontSide = getSonars()
    #move(0,0)    
    # turn until the wall is detected by sonar 0
    while leftSide > MAX_WALL_THRESHOLD:
        if frontSide < p3dx.distance[7] :
            yawSpeed = DEF_YAW_SPEED * 3
        else:
            yawSpeed = DEF_YAW_SPEED
        move(0,-yawSpeed)
        leftSide, frontSide = getSonars()
    move(0,0)

In [372]:
def is_line_detected():
    cx, cy, line_end = line_centroid(p3dx.image)
    if cx != None:
        return True
    else:
        return False

In [373]:
def follow_wall():
    print('Following the wall')
    while not is_line_detected():
        leftSide, frontSide = getSonars()
        # by default, just move forward
        xSpeed = DEF_X_SPEED
        yawSpeed = 0
        # if we're getting too close to the wall with the front side...
        if frontSide < MIN_WALL_THRESHOLD:
            # go backward and turn left quickly (x4)
            xSpeed = -0.1
            yawSpeed = DEF_YAW_SPEED * 4
        else:
            # if we're getting too close to the wall with the left side...
            if leftSide < MIN_WALL_THRESHOLD:
                # move slowly forward (x0.5) and turn left
                xSpeed  = DEF_X_SPEED * 0.5
                yawSpeed = DEF_YAW_SPEED
            else:
                # if we're getting too far away from the wall with the right side...
                if leftSide > MAX_WALL_THRESHOLD:
                    # move slowly forward (x0.5) and turn left
                    xSpeed  = DEF_X_SPEED * 0.8
                    yawSpeed = -DEF_YAW_SPEED
        # Move the robot
        move(xSpeed,-yawSpeed)
    print('Line detected')

In [374]:
def getLine():
    width = 150
    Kp = 0.1
    Kpp = 0.2
    cx,cy, line_end = line_centroid(p3dx.image)
    err = cx - width/2
    move(0,0)
    if is_line_detected():
        err = cx - width/2
        while err > 1:
            linear = Kpp*err
            angular = -Kp*err
            move(linear, angular)
            cx,cy = line_centroid(p3dx.image)
            err = cx - width/2

In [ ]:
# Lower-level functions
def line_centroid(image):
    hsv = cv2.cvtColor(p3dx.image, cv2.COLOR_RGB2HSV)
    lower_purple = numpy.array([135, 50, 50])
    upper_purple = numpy.array([165, 255, 255])
    
    lower_green = numpy.array([50, 50, 50])
    upper_green = numpy.array([70, 255, 255])
    
    mask_purple = cv2.inRange(hsv, lower_purple, upper_purple)
    mask_purple[0:80, 0:150] = 0
    
    mask_green = cv2.inRange(hsv, lower_green, upper_green)
    mask_green[0:80, 0:150] = 0
    
    M_purple = cv2.moments(mask_purple)
    
    M_green = cv2.moments(mask_green)
    
    line_end = False
    
    if M_purple['m00'] != 0 :
        if M_green ['m00'] == 0 :
            cx = int(M_purple['m10']/M_purple['m00'])
            cy = int(M_purple['m01']/M_purple['m00'])
        else:
            cx = (int(M_purple['m10']/M_purple['m00']) + int(M_green['m10']/M_green['m00'])) / 2
            cy = (int(M_purple['m01']/M_purple['m00']) + int(M_green['m01']/M_green['m00'])) / 2
            
    else:
        if M_green ['m00'] == 0 :
            cx = None
            cy = None
        else:
            cx = None
            cy = None
            line_end = True
    return cx, cy, line_end

def move(V_robot,w_robot):
    r = 0.1953 / 2
    L = 0.33
    w_r = (2 * V_robot + L * w_robot) / (2*r)
    w_l = (2 * V_robot - L * w_robot) / (2*r)
    p3dx.move(w_l, w_r)
    
def getSonars():
    leftSide = p3dx.distance[0]
    #rightSide = p3dx.distance[7]
    frontSide = min(p3dx.distance[3:5])
    return leftSide, frontSide

In [ ]:
p3dx.tilt(-0.4)
try:
    while True:
        follow_line()
        getWall()
        follow_wall()
        getLine()
except KeyboardInterrupt:
    move(0,0)

Following the line
Obstacle detected
Following the wall


---
#### Try-a-Bot: an open source guide for robot programming
Developed by:
[![Robotic Intelligence Lab @ UJI](img/logo/robinlab.png "Robotic Intelligence Lab @ UJI")](http://robinlab.uji.es)

Sponsored by:
<table>
<tr>
<td style="border:1px solid #ffffff ;">
<a href="http://www.ieee-ras.org"><img src="img/logo/ras.png"></a>
</td>
<td style="border:1px solid #ffffff ;">
<a href="http://www.cyberbotics.com"><img src="img/logo/cyberbotics.png"></a>
</td>
<td style="border:1px solid #ffffff ;">
<a href="http://www.theconstructsim.com"><img src="img/logo/theconstruct.png"></a>
</td>
</tr>
</table>

Follow us:
<table>
<tr>
<td style="border:1px solid #ffffff ;">
<a href="https://www.facebook.com/RobotProgrammingNetwork"><img src="img/logo/facebook.png"></a>
</td>
<td style="border:1px solid #ffffff ;">
<a href="https://www.youtube.com/user/robotprogrammingnet"><img src="img/logo/youtube.png"></a>
</td>
</tr>
</table>